In [ ]:
from __future__ import division
import pyomo.environ as pyomo
import pyomo.opt as opt

# Welcome to the TESA programming exercise on basic dispatch LP models

## 1. Prepare the input data for the LP model

In [ ]:
# introduce sets of indices
I = ['Seattle','San Diego'] # set of indices for supply
J = ['Topeka','Chicago','New York'] # set of indices for demand

In [ ]:
# supply amounts
supply_i = {'Seattle':350, 'San Diego':600}

In [ ]:
# demand  
demand_j = {'Topeka':275, 'Chicago':300, 'New York': 325}

In [ ]:
# shipping costs
costs_i_j = {('Seattle','Topeka'):  162, ('Seattle','Chicago'):  153, ('Seattle','New York'):  234, 
             ('San Diego','Topeka'):126, ('San Diego','Chicago'):162, ('San Diego','New York'):189}

In [ ]:
# min supply 
min_supply = 100

## 2. Build the LP model

In [ ]:
model = pyomo.ConcreteModel()

### 2.1 Define Sets

In [ ]:
# define sets of the LP model  
model.I = pyomo.Set(initialize=I)
model.J = pyomo.Set(initialize=J)

### 2.2 Define Variables

In [ ]:
# create decision variables
model.x_i_j = pyomo.Var(model.I, model.J, domain=pyomo.NonNegativeIntegers) # electricity output of technology g [TWh]

### 2.3 Define Constraints

In [ ]:
# cover demand
def define_demand_restriction(model, j):
    return sum(model.x_i_j[i,j] for i in model.I) == demand_j[j]
model.demand_restriction = pyomo.Constraint(model.J, rule=define_demand_restriction)

In [ ]:
# supply capacity restrictions
def define_capacity_restriciton(model, i):
    return sum(model.x_i_j[i,j] for j in model.J) <= supply_i[i]
model.capacity_restriciton = pyomo.Constraint(model.I, rule=define_capacity_restriciton)

In [ ]:
# minimum supply restrictions 
def define_minimum_supply_restriction(model, i, j):
    return model.x_i_j[i,j] >= min_supply
model.minimum_supply_restriction = pyomo.Constraint(model.I, model.J, rule=define_minimum_supply_restriction)

### 2.4 Define Objective Function

In [ ]:
# define objective function
def define_objective_function(model):
    return sum(costs_i_j[i,j] * model.x_i_j[i,j] for i in model.I for j in model.J)
model.Obj = pyomo.Objective(rule=define_objective_function, sense=pyomo.minimize)

### 2.4 Write LP to File

In [ ]:
model.write('output/transport/04_esa_uebung_LP1_transport_loesung_min_menge.lp', io_options={'symbolic_solver_labels':True})

## 3. Solve the LP model

In [ ]:
# call solver
optimizer = opt.SolverFactory('glpk')
solved_model = optimizer.solve(model, tee=True)

## 4. Get the results and statistics of the solved LP model

### 4.1 print optimal objective value

In [ ]:
print("Optimal value: %i" % (round(model.Obj.expr(),2)))

### 4.2 print optimal dispatch path

In [ ]:
EPS = 1.e-6
for i in model.I:
    for j in model.J:
        if pyomo.value(model.x_i_j[i,j]) > EPS:
            print("sending quantity %i from %s to %s" % (pyomo.value(model.x_i_j[i,j]),i,j))